# EPH - etiquetado
Etiquetaremos las bases de EPH con un archivo JSON con las etiquetas y un pequeño código en python.

________________________________________________
Cargamos la EPH con la función disponible [aquí]((https://github.com/holatam/eph/blob/e231da37b962e474febc06a96007604150cbb6ec/R/python/epyh.py)) en el paquete de [EPH para R](https://github.com/holatam/eph)

Hacemos unas pequeñas correcciones a la función, ya que la página de INDEC cambió una letra de sus archivos :D

In [32]:
import requests, zipfile, io
import pandas as pd
import json

ruta_etiquetas_individual = 'bases\eph_etiquetas_individual_valores.txt'
ruta_etiquetas_hogar = 'bases\eph_etiquetas_hogar_valores.txt'

def epyh(anio = 2018, trimestre = 2,individual = False, etiquetar=True):

    url='https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_{}_Trim_{}_txt.zip'.format(str(trimestre),str(anio))

    #decarga zip
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    
    if individual:
        if anio==2020: # T mayúscula para 2020
            nombre_archivo = 'usu_individual_T{}{}.txt'.format(str(trimestre),str(anio)[2:])
        else: 
            nombre_archivo = 'usu_individual_t{}{}.txt'.format(str(trimestre),str(anio)[2:])
        # cargo etiquetas
        with open(ruta_etiquetas_individual) as json_data:
            etiquetas = json.load(json_data)
        json_data.close()
    
    else:
        if anio==2020:
            nombre_archivo = 'usu_hogar_T{}{}.txt'.format(str(trimestre),str(anio)[2:])
        else:
            nombre_archivo = 'usu_hogar_t{}{}.txt'.format(str(trimestre),str(anio)[2:])

        with open(ruta_etiquetas_hogar) as json_data:
            etiquetas = json.load(json_data)
        json_data.close()

    #leer datos
    dt = pd.read_csv(z.open(nombre_archivo),sep=';',dtype=str)
    
    # etiquetar
    if etiquetar:
        dt = dt.replace(etiquetas)

    return dt

In [33]:
df = epyh(anio = 2020, trimestre = 1,individual = True,etiquetar=True)
print(df.shape)
df.head()

(51643, 177)


,CODUSU,ANO4,TRIMESTRE,NRO_HOGAR,COMPONENTE,H15,REGION,MAS_500,AGLOMERADO,PONDERA,...,PDECIFR,ADECIFR,IPCF,DECCFR,IDECCFR,RDECCFR,GDECCFR,PDECCFR,ADECCFR,PONDIH
0,TQRMNORVSHJOLPCDEGOIH00625031,2020,1er Trimestre,1,5,Si,Noroeste,NaN,S. del Estero - La Banda,230,...,NaN,06,"5406,25",01,01,02,NaN,NaN,02,230
1,TQRMNORVSHJOLPCDEGOIH00625031,2020,1er Trimestre,1,6,Si,Noroeste,NaN,S. del Estero - La Banda,230,...,NaN,06,"5406,25",01,01,02,NaN,NaN,02,230
2,TQRMNORVSHJOLPCDEGOIH00625031,2020,1er Trimestre,1,7,0,Noroeste,NaN,S. del Estero - La Banda,230,...,NaN,06,"5406,25",01,01,02,NaN,NaN,02,230
3,TQRMNORVSHJOLPCDEGOIH00625031,2020,1er Trimestre,1,8,0,Noroeste,NaN,S. del Estero - La Banda,230,...,NaN,06,"5406,25",01,01,02,NaN,NaN,02,230
4,TQSMNOQRQHLMLPCDEGOIH00654347,2020,1er Trimestre,1,1,Si,Noroeste,NaN,S. del Estero - La Banda,182,...,NaN,02,6500,02,02,02,NaN,NaN,02,181
